In [1]:
import gym
import numpy as np

env = gym.make('Blackjack-v1')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
Q = np.zeros((32, 11, 2, 2))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
num_episodes = 100000
learning_rate = 0.1
discount_factor = 0.99
epsilon = 0.1

# Using Q-learning

In [4]:
for episode in range(num_episodes):
    state = env.reset()
    done = False

    while not done:
        player_sum, dealer_card, usable_ace = state
        player_sum -= 12
        dealer_card -= 1
        usable_ace = int(usable_ace)


        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[player_sum, dealer_card, usable_ace, :])

        next_state, reward, done, _ = env.step(action)


        old_q_value = Q[player_sum, dealer_card, usable_ace, action]
        next_max_q_value = np.max(Q[next_state[0]-12, next_state[1]-1, int(next_state[2]), :])
        new_q_value = old_q_value + learning_rate * (reward + discount_factor * next_max_q_value - old_q_value)

        Q[player_sum, dealer_card, usable_ace, action] = new_q_value

        state = next_state

In [5]:
num_episodes = 10000
wins = 0
loss = 0
draw = 0
for episode in range(num_episodes):
    state = env.reset()
    done = False

    while not done:
        player_sum, dealer_card, usable_ace = state
        player_sum -= 12
        dealer_card -= 1
        usable_ace = int(usable_ace)

        action = np.argmax(Q[player_sum, dealer_card, usable_ace, :])

        next_state, reward, done, _ = env.step(action)

        state = next_state

    if reward == 1:
        wins += 1
    elif reward == -1:
        loss += 1
    else:
        draw += 1


print("Wins:", wins)
print("Losses:", loss)
print("Draws:", draw)

Wins: 3915
Losses: 5383
Draws: 702


In [6]:
win_percentage = (wins / num_episodes) * 100
loss_percentage = (loss / num_episodes) * 100
draw_percentage = (draw / num_episodes) * 100
print("Win Percentage: %.2f%%" % win_percentage)
print("Loss Percentage: %.2f%%" % loss_percentage)
print("Draw Percentage: %.2f%%" % draw_percentage)

Win Percentage: 39.15%
Loss Percentage: 53.83%
Draw Percentage: 7.02%


# Using SARSA

In [7]:
Q = np.zeros((32, 11, 2, 2))

In [11]:
for episode in range(num_episodes):
    state = env.reset()
    done = False


    player_sum, dealer_card, usable_ace = state
    player_sum -= 12
    dealer_card -= 1
    usable_ace = int(usable_ace)
    if np.random.rand() < epsilon:
        action = env.action_space.sample()
    else:
        action = np.argmax(Q[player_sum, dealer_card, usable_ace, :])

    while not done:
        next_state, reward, done, _ = env.step(action)

        next_player_sum, next_dealer_card, next_usable_ace = next_state
        next_player_sum -= 12
        next_dealer_card -= 1
        next_usable_ace = int(next_usable_ace)

        if np.random.rand() < epsilon:
            next_action = env.action_space.sample()
        else:
            next_action = np.argmax(Q[next_player_sum, next_dealer_card, next_usable_ace, :])


        current_q_value = Q[player_sum, dealer_card, usable_ace, action]
        next_q_value = Q[next_player_sum, next_dealer_card, next_usable_ace, next_action]
        updated_q_value = current_q_value + learning_rate * (reward + discount_factor * next_q_value - current_q_value)


        Q[player_sum, dealer_card, usable_ace, action] = updated_q_value


        state = next_state
        action = next_action

In [9]:
num_episodes = 10000
wins = 0
loss = 0
draw = 0
for episode in range(num_episodes):
    state = env.reset()
    done = False

    while not done:
        player_sum, dealer_card, usable_ace = state
        player_sum -= 12
        dealer_card -= 1
        usable_ace = int(usable_ace)

        action = np.argmax(Q[player_sum, dealer_card, usable_ace, :])

        next_state, reward, done, _ = env.step(action)

        state = next_state

    if reward == 1:
        wins += 1
    elif reward == -1:
        loss += 1
    else:
        draw += 1

print("Wins:", wins)
print("Losses:", loss)
print("Draws:", draw)

Wins: 3939
Losses: 5356
Draws: 705


In [10]:
win_percentage = (wins / num_episodes) * 100
loss_percentage = (loss / num_episodes) * 100
draw_percentage = (draw / num_episodes) * 100
print("Win Percentage: %.2f%%" % win_percentage)
print("Loss Percentage: %.2f%%" % loss_percentage)
print("Draw Percentage: %.2f%%" % draw_percentage)

Win Percentage: 39.39%
Loss Percentage: 53.56%
Draw Percentage: 7.05%
